In [1]:
# import packages
import pandas as pd
from gurobipy import *
import numpy as np

# read excel sheets with pandas
data = pd.ExcelFile('SuperChipData.xlsx')
productionCapacityDf = pd.read_excel(data, 'Production Capacity')
salesRegionDemandDf = pd.read_excel(data, 'Sales Region Demand')
shippingCostsDf = pd.read_excel(data, 'Shipping Costs')
productionCostsDf = pd.read_excel(data, 'Production Costs')

# preparing for adding variables
# total number of sales region j
salesRegion = 23
# total number of computer chip z
computerChip = 30
# [sales region, computer chip, yearly demand(thousands)]
salesRegionDemand = salesRegionDemandDf.to_numpy()
# [facility name, facility yearly production capacity(thousands)]
productionCapacity = productionCapacityDf.to_numpy()
# [facility, computer chip, production cost per chip ($)]
productionCosts = productionCostsDf.to_numpy()
# [facility, computer chip, sales region, shipping cost per chip($)]
shippingCosts = shippingCostsDf.to_numpy()

# initialize model
m = Model()
# set objective function to be minimization
m.modelSense = GRB.MINIMIZE

# declare variables and add them to model
# facility set S
facilitySet = ["Alexandria", "Richmond", "Norfolk", "Roanoke", "Charolottesville"]
# assign number to facility
facilityDict = {}
for i, f in enumerate(facilitySet):
     facilityDict[f] = i + 1

# Djz = z chip yearly demand in sales region j (thousands unit)
D = {}
for d in salesRegionDemand:
        # (j, z) = demand in j z
        D[d[0], d[1]] = d[2]

# Ci = yearly chip production capacity in facility i (thousands unit)
C = {}
for c in productionCapacity:
     # i = production capacity in i
     C[facilityDict[c[0]]] = c[1]

# Piz = production cost per z chip in facility i ($)
P = {}
for p in productionCosts:
     # (i, z) = production cost in i, z
     P[facilityDict[p[0]], p[1]] = p[2]

# Sijz = shipping cost per z chip from facility i to sales region j ($)
S = {}
for s in shippingCosts:
     # (i, j, z) = shipping cost per z from i to j
     S[facilityDict[s[0]], s[2], s[1]] = s[3]

# Xijz = amount of z chips shipped from facility i to sales region j (unit)
X = {}
# iterate through facility set, sales region numbers and computer chip numbers
for i in facilitySet:
    for j in range(1, salesRegion + 1):
        for z in range(1, computerChip + 1):
            X[facilityDict[i], j, z] = m.addVar(lb = 0, vtype = GRB.CONTINUOUS, 
                                                name = f"X_{i}_{j}_{z}")

# Yiz = amount of z chips produced in facility i (unit)
Y = {}
# iterate through facility set, computer chip numbers
for i in facilitySet:
    for z in range(1, computerChip + 1):
        Y[facilityDict[i], z] = m.addVar(lb = 0, vtype = GRB.CONTINUOUS, 
                                         name = f"Y_{i}_{z}")
        
# notify model the changes
m.update()

# set the objective function
m.setObjective(quicksum(S[facilityDict[i], j, z] * X[facilityDict[i], j, z] for i 
                        in facilitySet for j in range(1, salesRegion + 1) for z in 
                        range(1, computerChip + 1)) + 
               quicksum(P[facilityDict[i], z] * Y[facilityDict[i], z] for i 
                        in facilitySet for z in range(1, computerChip + 1)))

# first constraint ∑Yiz <= 1000Ci
facilityProductionConst = {}
for i in facilitySet:
     facilityProductionConst[i] = m.addConstr(quicksum(
          Y[facilityDict[i], z] for z in range(1, computerChip + 1)
          ) <= 1000 * C[facilityDict[i]], name = f"production_{i}")

# second constraint ∑Xijz = 1000Djz
shipDemandConst = {}
for j in range(1, salesRegion + 1):
     for z in range(1, computerChip + 1):
          shipDemandConst[j, z] = m.addConstr(quicksum(
               X[facilityDict[i], j, z] for i in facilitySet) == 1000 * D[j, z],
               name = f"shipping_demand_{j}_{z}")

# third constraint Yiz >= ∑Xijz
productionShipConst = {}
for i in facilitySet:
     for z in range(1, computerChip + 1):
          productionShipConst[facilityDict[i], z] = m.addConstr(
               Y[facilityDict[i], z] >= quicksum(
                    X[facilityDict[i], j, z] for j in range(1, salesRegion + 1)), 
                    name = f"production_shipping_{i}_{z}")

# for the fourth and fifth constraints, since Xijz and Yiz both have lower bound of
# 0 on initialization, not adding them here

# notify model the changes
m.update()

# trigger optimization
m.optimize()

# print the object function value
print ("Optimal Production and Distribution Cost = ", m.objVal)
# write model to file
m.write("Computer-Chip-Project-Model.lp")
# write solution to file
m.write("Computer-Chip-Project-Model.sol")



Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-25
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 22.3.0 22D49)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 845 rows, 3600 columns and 7200 nonzeros
Model fingerprint: 0x756da770
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+05]
Presolve removed 151 rows and 155 columns
Presolve time: 0.00s
Presolved: 694 rows, 3445 columns, 6890 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9059636e+07   7.737500e+03   0.000000e+00      0s
      45    4.9083430e+07   0.000000e+00   0.000000e+00      0s

Solved in 45 iterations and 0.01 seconds (0.01 work units)
Optimal objective  4.908343040e+07
Optimal Production and Distribution Cost =  49083430.39999999


In [2]:
# To answer question 1 - 

# find out the total production capacity in all facilities
# initialize
totalProductionCapacity = 0
# add it up
for c in productionCapacity:
    totalProductionCapacity += c[1]
# in thousand unit
totalProductionCapacity *= 1000

# lookup map to store facility x has 
# y% of the total production capacity 
facilityPercentageDict = {}
for f, p in productionCapacity:
    facilityPercentageDict[f] = (p * 1000) / totalProductionCapacity
print(facilityPercentageDict)

# every chip's total demand
chipDemandDict = {}
for i, j, z in salesRegionDemand:
    if j not in chipDemandDict:
        chipDemandDict[j] = 0
    else:
        chipDemandDict[j] += (1000 * z)
print(chipDemandDict)

# the current policy cost -
currentPolicyProductionCost = 0
# amount of chips produced in facility on y%
chipsProducedInFacilityDict = {}
for i in facilitySet:
    for z in range(1, computerChip + 1):  
        # facility i produces y% of every chip's total demand
        chipsProducedInFacilityDict[(i, z)] = chipDemandDict[z] * facilityPercentageDict[i]
# calculate total policy cost based on facility chip production cost
for i, z, c in productionCosts:
    if (i, z) in chipsProducedInFacilityDict:
        currentPolicyProductionCost += chipsProducedInFacilityDict[(i, z)] * c
print("current policy production cost ($): ", currentPolicyProductionCost)

newPolicyProductionCost = 0
for i, z, c in productionCosts:
    if (facilityDict[i], z) in Y:
        newPolicyProductionCost += Y[facilityDict[i], z].x * c
print("new policy production cost ($): ", newPolicyProductionCost)
print("production policy costs difference ($): ", abs(
    currentPolicyProductionCost - newPolicyProductionCost))

print("new policy facility production percentage per chip:") 
# find out the new policy production percentage based
# on optimal objective function value -
facilityChipProductionDict = {}
for i in facilitySet:
    for z in range(1, computerChip + 1):
        # find out current percentage based on chip's total demand
        facilityChipProductionDict[(i, z)] = Y[facilityDict[i], z].x / chipDemandDict[z]
        print(f"{i}_{z}:", facilityChipProductionDict[(i, z)])


{'Alexandria': 0.25327510917030566, 'Richmond': 0.22707423580786026, 'Norfolk': 0.21397379912663755, 'Roanoke': 0.1572052401746725, 'Charolottesville': 0.14847161572052403}
{1.0: 29420.0, 2.0: 39160.0, 3.0: 39770.0, 4.0: 33330.0, 5.0: 34170.0, 6.0: 32230.0, 7.0: 33080.0, 8.0: 34710.0, 9.0: 32290.0, 10.0: 30160.0, 11.0: 30930.0, 12.0: 29860.0, 13.0: 29610.0, 14.0: 35680.0, 15.0: 29870.0, 16.0: 34770.0, 17.0: 35250.0, 18.0: 35360.0, 19.0: 30290.0, 20.0: 32220.0, 21.0: 35630.0, 22.0: 35880.0, 23.0: 32880.0, 24.0: 31150.0, 25.0: 28440.0, 26.0: 29410.0, 27.0: 39770.0, 28.0: 32610.0, 29.0: 37420.0, 30.0: 32490.0}
current policy production cost ($):  52084233.8235808
new policy production cost ($):  47309680.39999999
production policy costs difference ($):  4774553.42358081
new policy facility production percentage per chip:
Alexandria_1: 0.0
Alexandria_2: 0.0
Alexandria_3: 0.0
Alexandria_4: 0.0
Alexandria_5: 0.0
Alexandria_6: 0.7638845795842383
Alexandria_7: 0.0
Alexandria_8: 0.8827427254393

In [3]:
# To answer question 2 - 

# function to calculate total shipping cost per facility
def total_shipping_cost_per_facility(costDict, shippingCosts, X):
    for i, z, j, c in shippingCosts:
        if i not in costDict:
            costDict[i] = 0
        else:
            costDict[i] += (X[facilityDict[i], j, z].x * c)
    return costDict

# function to calculate total production cost per facility
def total_production_cost_per_facility(costDict, productionCosts, Y):
    for i, z, c in productionCosts:
        if i not in costDict:
            costDict[i] = 0
        else:
            costDict[i] += (Y[facilityDict[i], z].x * c)
    return costDict

# helper function to increase facility capacity one by one for analyses
def capacity_increase_helper(increaseAmount, capacityArr, i, j):
    newCapacity = np.copy(capacityArr)
    newCapacity[i][j] += increaseAmount
    return newCapacity

# generate new Ci = yearly chip production capacity in facility i (thousands unit)
# after the capacity increase
def chip_production_per_facility(newCapacityArr, capacityDict):
    for c in newCapacityArr:
         # i = production capacity in i
         capacityDict[facilityDict[c[0]]] = c[1]
    return capacityDict

# generate model with respective new facility capacity
def model_builder(m, X, Y, capacityDict, facilityProductionConst, shipDemandConst, productionShipConst):
    # set objective function to be minimization
    m.modelSense = GRB.MINIMIZE

    # iterate through facility set, sales region numbers and computer chip numbers
    for i in facilitySet:
        for j in range(1, salesRegion + 1):
            for z in range(1, computerChip + 1):
                X[facilityDict[i], j, z] = m.addVar(lb = 0, vtype = GRB.CONTINUOUS, 
                                                    name = f"X_{i}_{j}_{z}")

    # iterate through facility set, computer chip numbers
    for i in facilitySet:
        for z in range(1, computerChip + 1):
            Y[facilityDict[i], z] = m.addVar(lb = 0, vtype = GRB.CONTINUOUS, 
                                            name = f"Y_{i}_{z}")

    # notify model the changes
    m.update()

    # set the objective function
    m.setObjective(quicksum(S[facilityDict[i], j, z] * X[facilityDict[i], j, z] for i 
                            in facilitySet for j in range(1, salesRegion + 1) for z in 
                            range(1, computerChip + 1)) + 
                quicksum(P[facilityDict[i], z] * Y[facilityDict[i], z] for i 
                            in facilitySet for z in range(1, computerChip + 1)))

    for i in facilitySet:
        facilityProductionConst[i] = m.addConstr(quicksum(
            Y[facilityDict[i], z] for z in range(1, computerChip + 1)
            ) <= 1000 * capacityDict[facilityDict[i]], name = f"production_{i}")
        
    for j in range(1, salesRegion + 1):
        for z in range(1, computerChip + 1):
            shipDemandConst[j, z] = m.addConstr(quicksum(
                X[facilityDict[i], j, z] for i in facilitySet) == 1000 * D[j, z],
                name = f"shipping_demand_{j}_{z}")

    for i in facilitySet:
        for z in range(1, computerChip + 1):
            productionShipConst[facilityDict[i], z] = m.addConstr(
                Y[facilityDict[i], z] >= quicksum(
                        X[facilityDict[i], j, z] for j in range(1, salesRegion + 1)), 
                        name = f"production_shipping_{i}_{z}")

    # for the fourth and fifth constraints, since Xijz and Yiz both have lower bound of
    # 0 on initialization, not adding them here
    # notify model the changes
    m.update()
    # trigger optimization
    m.optimize()
    # print the object function value
    print("Optimal Production and Distribution Costs = ", m.objVal)

# calculate the change in shipping/production cost after capacity increase in a facility
def cost_difference_calculator(oldTotalCost, newTotalCost):
    totalDiff = {}
    for k, v in oldTotalCost.items():
        if k not in totalDiff:
            totalDiff[k] = 0
        # if positive meaning it is more suboptimal, negative meaning it is more optimal
        totalDiff[k] += newTotalCost[k] - v
    return totalDiff

# calculate objective function value change before and after capacity increase in a facility
# if positive meaning it is more suboptimal, negative meaning it is more optimal
def objective_function_value_change(before, after):
    return after - before

In [4]:
# optimal model total shipping cost per facility
optimalFacilityTotalShippingCost = total_shipping_cost_per_facility({}, shippingCosts, X)

# optimal model total production cost per facility
optimalFacilityTotalProductionCost = total_production_cost_per_facility({}, productionCosts, Y)

# Increase Alexandria facility capacity by 50 (thousands)
ANewCapacity = capacity_increase_helper(50, productionCapacity, 0, 1)
CA = chip_production_per_facility(ANewCapacity, {})
print("production capacitiy after increase 50: ", CA)

mA = Model()
XA = {}
YA = {}
AFacilityProductionConst = {}
AShipDemandConst = {}
AProductionShipConst = {}
ACapacityIncreaseModel = model_builder(mA, XA, YA, CA, AFacilityProductionConst, AShipDemandConst, AProductionShipConst)

# calculate new shipping cost difference
ANewShippingCost = total_shipping_cost_per_facility({}, shippingCosts, XA)
ATotalShippingCostChange = cost_difference_calculator(optimalFacilityTotalShippingCost, ANewShippingCost)
print("Total Shipping Cost change in Alexandria Facility: ", ATotalShippingCostChange)

# calculate new production cost difference
ANewProductionCost = total_production_cost_per_facility({}, productionCosts, YA)
ATotalProductionCostChange = cost_difference_calculator(optimalFacilityTotalProductionCost, ANewProductionCost)
print("Total Production Cost change in Alexandria Facility: ", ATotalProductionCostChange)

# calculate objective value change
AObjValChange = objective_function_value_change(m.ObjVal, mA.ObjVal)
print("Change in Objective Function Value in Alexandria Facility: ", AObjValChange)


# Increase Alexandria facility capacity by 100 (thousands)
A1NewCapacity = capacity_increase_helper(100, productionCapacity, 0, 1)
CA1 = chip_production_per_facility(A1NewCapacity, {})
print("production capacitiy after increase 100: ", CA1)

mA1 = Model()
XA1 = {}
YA1 = {}
A1FacilityProductionConst = {}
A1ShipDemandConst = {}
A1ProductionShipConst = {}
A1CapacityIncreaseModel = model_builder(mA1, XA1, YA1, CA1, A1FacilityProductionConst, A1ShipDemandConst, A1ProductionShipConst)

# calculate new shipping cost difference
A1NewShippingCost = total_shipping_cost_per_facility({}, shippingCosts, XA1)
A1TotalShippingCostChange = cost_difference_calculator(optimalFacilityTotalShippingCost, A1NewShippingCost)
print("Total Shipping Cost change1 in Alexandria Facility: ", A1TotalShippingCostChange)

# calculate new production cost difference
A1NewProductionCost = total_production_cost_per_facility({}, productionCosts, YA1)
A1TotalProductionCostChange = cost_difference_calculator(optimalFacilityTotalProductionCost, A1NewProductionCost)
print("Total Production Cost change1 in Alexandria Facility: ", A1TotalProductionCostChange)

# calculate objective value change
A1ObjValChange = objective_function_value_change(m.ObjVal, mA1.ObjVal)
print("Change in Objective Function Value in Alexandria Facility: ", A1ObjValChange)

production capacitiy after increase 50:  {1: 398, 2: 312, 3: 294, 4: 216, 5: 204}
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 22.3.0 22D49)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 845 rows, 3600 columns and 7200 nonzeros
Model fingerprint: 0x95ca7e48
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 4e+05]
Presolve removed 151 rows and 155 columns
Presolve time: 0.00s
Presolved: 694 rows, 3445 columns, 6890 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9059636e+07   7.737500e+03   0.000000e+00      0s
      45    4.9083430e+07   0.000000e+00   0.000000e+00      0s

Solved in 45 iterations and 0.01 seconds (0.01 work units)
Optimal objective  4.908343040e+07
Optimal Production and Distribution Costs =  49083430.39999999
Total Shippin

In [5]:
# Increase Richmond facility capacity by 50 (thousands)
RNewCapacity = capacity_increase_helper(50, productionCapacity, 1, 1)
CR = chip_production_per_facility(RNewCapacity, {})
print("production capacitiy after increase 50: ", CR)

mR = Model()
XR = {}
YR = {}
RFacilityProductionConst = {}
RShipDemandConst = {}
RProductionShipConst = {}
RCapacityIncreaseModel = model_builder(mR, XR, YR, CR, RFacilityProductionConst, RShipDemandConst, RProductionShipConst)

# calculate new shipping cost difference
RNewShippingCost = total_shipping_cost_per_facility({}, shippingCosts, XR)
# calculate new production cost difference
RNewProductionCost = total_production_cost_per_facility({}, productionCosts, YR)

RTotalShippingCostChange = cost_difference_calculator(optimalFacilityTotalShippingCost, RNewShippingCost)
print("Total Shipping Cost change in Richmond Facility: ", RTotalShippingCostChange)

RTotalProductionCostChange = cost_difference_calculator(optimalFacilityTotalProductionCost, RNewProductionCost)
print("Total Production Cost change in Richmond Facility: ", RTotalProductionCostChange)

# calculate objective value change
RObjValChange = objective_function_value_change(m.ObjVal, mR.ObjVal)
print("Change in Objective Function Value in Richmond Facility: ", RObjValChange)


# Increase Richmond facility capacity by 100 (thousands)
R1NewCapacity = capacity_increase_helper(100, productionCapacity, 1, 1)
CR1 = chip_production_per_facility(R1NewCapacity, {})
print("production capacitiy after increase 100: ", CR1)

mR1 = Model()
XR1 = {}
YR1 = {}
R1FacilityProductionConst = {}
R1ShipDemandConst = {}
R1ProductionShipConst = {}
R1CapacityIncreaseModel = model_builder(mR1, XR1, YR1, CR1, R1FacilityProductionConst, R1ShipDemandConst, R1ProductionShipConst)

# calculate new shipping cost difference
R1NewShippingCost = total_shipping_cost_per_facility({}, shippingCosts, XR1)
# calculate new production cost difference
R1NewProductionCost = total_production_cost_per_facility({}, productionCosts, YR1)

R1TotalShippingCostChange = cost_difference_calculator(optimalFacilityTotalShippingCost, R1NewShippingCost)
print("Total Shipping Cost change1 in Richmond Facility: ", R1TotalShippingCostChange)

R1TotalProductionCostChange = cost_difference_calculator(optimalFacilityTotalProductionCost, R1NewProductionCost)
print("Total Production Cost change1 in Richmond Facility: ", R1TotalProductionCostChange)

# calculate objective value change
R1ObjValChange = objective_function_value_change(m.ObjVal, mR1.ObjVal)
print("Change1 in Objective Function Value in Richmond Facility: ", R1ObjValChange)

production capacitiy after increase 50:  {1: 348, 2: 362, 3: 294, 4: 216, 5: 204}
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 22.3.0 22D49)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 845 rows, 3600 columns and 7200 nonzeros
Model fingerprint: 0x4c8231ec
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 4e+05]
Presolve removed 151 rows and 155 columns
Presolve time: 0.00s
Presolved: 694 rows, 3445 columns, 6890 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9059636e+07   1.487500e+03   0.000000e+00      0s
       8    4.9060386e+07   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds (0.01 work units)
Optimal objective  4.906038600e+07
Optimal Production and Distribution Costs =  49060386.0
Total Shipping Cost c

In [6]:
# Increase Norfolk facility capacity by 50 (thousands)
NNewCapacity = capacity_increase_helper(50, productionCapacity, 2, 1)
CN = chip_production_per_facility(NNewCapacity, {})
print("production capacitiy after increase 50: ", CN)

mN = Model()
XN = {}
YN = {}
NFacilityProductionConst = {}
NShipDemandConst = {}
NProductionShipConst = {}
NCapacityIncreaseModel = model_builder(mN, XN, YN, CN, NFacilityProductionConst, NShipDemandConst, NProductionShipConst)

# calculate new shipping cost difference
NNewShippingCost = total_shipping_cost_per_facility({}, shippingCosts, XN)
# calculate new production cost difference
NNewProductionCost = total_production_cost_per_facility({}, productionCosts, YN)

NTotalShippingCostChange = cost_difference_calculator(optimalFacilityTotalShippingCost, NNewShippingCost)
print("Total Shipping Cost change in Norfolk Facility: ", NTotalShippingCostChange)

NTotalProductionCostChange = cost_difference_calculator(optimalFacilityTotalProductionCost, NNewProductionCost)
print("Total Production Cost change in Norfolk Facility: ", NTotalProductionCostChange)

# calculate objective value change
NObjValChange = objective_function_value_change(m.ObjVal, mN.ObjVal)
print("Change in Objective Function Value in Norfolk Facility: ", NObjValChange)

# Increase Norfolk facility capacity by 100 (thousands)
N1NewCapacity = capacity_increase_helper(100, productionCapacity, 2, 1)
CN1 = chip_production_per_facility(N1NewCapacity, {})
print("production capacitiy after increase 100: ", CN1)

mN1 = Model()
XN1 = {}
YN1 = {}
N1FacilityProductionConst = {}
N1ShipDemandConst = {}
N1ProductionShipConst = {}
NCapacityIncreaseModel = model_builder(mN1, XN1, YN1, CN1, N1FacilityProductionConst, N1ShipDemandConst, N1ProductionShipConst)

# calculate new shipping cost difference
N1NewShippingCost = total_shipping_cost_per_facility({}, shippingCosts, XN1)
# calculate new production cost difference
N1NewProductionCost = total_production_cost_per_facility({}, productionCosts, YN1)

N1TotalShippingCostChange = cost_difference_calculator(optimalFacilityTotalShippingCost, N1NewShippingCost)
print("Total Shipping Cost change1 in Norfolk Facility: ", N1TotalShippingCostChange)

N1TotalProductionCostChange = cost_difference_calculator(optimalFacilityTotalProductionCost, N1NewProductionCost)
print("Total Production Cost change1 in Norfolk Facility: ", N1TotalProductionCostChange)

# calculate objective value change
N1ObjValChange = objective_function_value_change(m.ObjVal, mN1.ObjVal)
print("Change1 in Objective Function Value in Norfolk Facility: ", N1ObjValChange)

production capacitiy after increase 50:  {1: 348, 2: 312, 3: 344, 4: 216, 5: 204}
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 22.3.0 22D49)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 845 rows, 3600 columns and 7200 nonzeros
Model fingerprint: 0x4e774d3b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+05]
Presolve removed 151 rows and 155 columns
Presolve time: 0.00s
Presolved: 694 rows, 3445 columns, 6890 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9059636e+07   7.737500e+03   0.000000e+00      0s
      45    4.9083430e+07   0.000000e+00   0.000000e+00      0s

Solved in 45 iterations and 0.01 seconds (0.01 work units)
Optimal objective  4.908343040e+07
Optimal Production and Distribution Costs =  49083430.39999999
Total Shippin

In [7]:
# Increase Roanoke facility capacity by 50 (thousands)
RoNewCapacity = capacity_increase_helper(50, productionCapacity, 3, 1)
CRo = chip_production_per_facility(RoNewCapacity, {})
print("production capacitiy after increase 50: ", CRo)

mRo = Model()
XRo = {}
YRo = {}
RoFacilityProductionConst = {}
RoShipDemandConst = {}
RoProductionShipConst = {}
RoCapacityIncreaseModel = model_builder(mRo, XRo, YRo, CRo, RoFacilityProductionConst, RoShipDemandConst, RoProductionShipConst)

# calculate new shipping cost difference
RoNewShippingCost = total_shipping_cost_per_facility({}, shippingCosts, XRo)
# calculate new production cost difference
RoNewProductionCost = total_production_cost_per_facility({}, productionCosts, YRo)

RoTotalShippingCostChange = cost_difference_calculator(optimalFacilityTotalShippingCost, RoNewShippingCost)
print("Total Shipping Cost change in Roanoke Facility: ", RoTotalShippingCostChange)

RoTotalProductionCostChange = cost_difference_calculator(optimalFacilityTotalProductionCost, RoNewProductionCost)
print("Total Production Cost change in Roanoke Facility: ", RoTotalProductionCostChange)

# calculate objective value change
RoObjValChange = objective_function_value_change(m.ObjVal, mRo.ObjVal)
print("Change in Objective Function Value in Roanoke Facility: ", RoObjValChange)

# Increase Roanoke facility capacity by 100 (thousands)
Ro1NewCapacity = capacity_increase_helper(100, productionCapacity, 3, 1)
CRo1 = chip_production_per_facility(Ro1NewCapacity, {})
print("production capacitiy after increas 100: ", CRo1)

mRo1 = Model()
XRo1 = {}
YRo1 = {}
Ro1FacilityProductionConst = {}
Ro1ShipDemandConst = {}
Ro1ProductionShipConst = {}
Ro1CapacityIncreaseModel = model_builder(mRo1, XRo1, YRo1, CRo1, Ro1FacilityProductionConst, Ro1ShipDemandConst, Ro1ProductionShipConst)

# calculate new shipping cost difference
Ro1NewShippingCost = total_shipping_cost_per_facility({}, shippingCosts, XRo1)
# calculate new production cost difference
Ro1NewProductionCost = total_production_cost_per_facility({}, productionCosts, YRo1)

Ro1TotalShippingCostChange = cost_difference_calculator(optimalFacilityTotalShippingCost, Ro1NewShippingCost)
print("Total Shipping Cost change1 in Roanoke Facility: ", RoTotalShippingCostChange)

Ro1TotalProductionCostChange = cost_difference_calculator(optimalFacilityTotalProductionCost, Ro1NewProductionCost)
print("Total Production Cost change1 in Roanoke Facility: ", RoTotalProductionCostChange)

# calculate objective value change
Ro1ObjValChange = objective_function_value_change(m.ObjVal, mRo1.ObjVal)
print("Change1 in Objective Function Value in Roanoke Facility: ", Ro1ObjValChange)

production capacitiy after increase 50:  {1: 348, 2: 312, 3: 294, 4: 266, 5: 204}
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 22.3.0 22D49)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 845 rows, 3600 columns and 7200 nonzeros
Model fingerprint: 0xad5ba122
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+05]
Presolve removed 151 rows and 155 columns
Presolve time: 0.00s
Presolved: 694 rows, 3445 columns, 6890 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9059636e+07   7.737500e+03   0.000000e+00      0s
      45    4.9083430e+07   0.000000e+00   0.000000e+00      0s

Solved in 45 iterations and 0.01 seconds (0.01 work units)
Optimal objective  4.908343040e+07
Optimal Production and Distribution Costs =  49083430.39999999
Total Shippin

In [8]:
# Increase Charolottesville facility capacity by 50 (thousands)
CNewCapacity = capacity_increase_helper(50, productionCapacity, 4, 1)
CC = chip_production_per_facility(CNewCapacity, {})
print("production capacitiy after increase: ", CC)

mC = Model()
XC = {}
YC = {}
CFacilityProductionConst = {}
CShipDemandConst = {}
CProductionShipConst = {}
CCapacityIncreaseModel = model_builder(mC, XC, YC, CC, CFacilityProductionConst, CShipDemandConst, CProductionShipConst)

# calculate new shipping cost difference
CNewShippingCost = total_shipping_cost_per_facility({}, shippingCosts, XC)
# calculate new production cost difference
CNewProductionCost = total_production_cost_per_facility({}, productionCosts, YC)

CTotalShippingCostChange = cost_difference_calculator(optimalFacilityTotalShippingCost, CNewShippingCost)
print("Total Shipping Cost change in Roanoke Facility: ", CTotalShippingCostChange)

CTotalProductionCostChange = cost_difference_calculator(optimalFacilityTotalProductionCost, CNewProductionCost)
print("Total Production Cost change in Roanoke Facility: ", CTotalProductionCostChange)

# calculate objective value change
CObjValChange = objective_function_value_change(m.ObjVal, mC.ObjVal)
print("Change in Objective Function Value in Roanoke Facility: ", CObjValChange)


# Increase Charolottesville facility capacity by 100 (thousands)
C1NewCapacity = capacity_increase_helper(100, productionCapacity, 4, 1)
C1C = chip_production_per_facility(C1NewCapacity, {})
print("production capacitiy after increas 100: ", C1C)

mC1 = Model()
XC1 = {}
YC1 = {}
C1FacilityProductionConst = {}
C1ShipDemandConst = {}
C1ProductionShipConst = {}
C1CapacityIncreaseModel = model_builder(mC1, XC1, YC1, C1C, C1FacilityProductionConst, C1ShipDemandConst, C1ProductionShipConst)

# calculate new shipping cost difference
C1NewShippingCost = total_shipping_cost_per_facility({}, shippingCosts, XC1)
# calculate new production cost difference
C1NewProductionCost = total_production_cost_per_facility({}, productionCosts, YC1)

C1TotalShippingCostChange = cost_difference_calculator(optimalFacilityTotalShippingCost, C1NewShippingCost)
print("Total Shipping Cost change1 in Roanoke Facility: ", C1TotalShippingCostChange)

C1TotalProductionCostChange = cost_difference_calculator(optimalFacilityTotalProductionCost, C1NewProductionCost)
print("Total Production Cost change1 in Roanoke Facility: ", C1TotalProductionCostChange)

# calculate objective value change
C1ObjValChange = objective_function_value_change(m.ObjVal, mC1.ObjVal)
print("Change1 in Objective Function Value in Roanoke Facility: ", C1ObjValChange)

production capacitiy after increase:  {1: 348, 2: 312, 3: 294, 4: 216, 5: 254}
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 22.3.0 22D49)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 845 rows, 3600 columns and 7200 nonzeros
Model fingerprint: 0xfac570f6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+05]
Presolve removed 151 rows and 155 columns
Presolve time: 0.00s
Presolved: 694 rows, 3445 columns, 6890 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9059636e+07   7.737500e+03   0.000000e+00      0s
      45    4.9083430e+07   0.000000e+00   0.000000e+00      0s

Solved in 45 iterations and 0.01 seconds (0.01 work units)
Optimal objective  4.908343040e+07
Optimal Production and Distribution Costs =  49083430.39999999
Total Shipping C

In [26]:
# To answer question 3 - 

# calculate total yearly demand per sales region
totalSalesRegionDemand = {}
for j, z, d in salesRegionDemand:
    if j not in totalSalesRegionDemand:
        totalSalesRegionDemand[j] = 0
    totalSalesRegionDemand[j] += d 

# demand increase by 10% across all sales regions
for k, v in totalSalesRegionDemand.items():
    totalSalesRegionDemand[k] = (0.1 * v) + v

# calculate difference between total production capacity and demand
capacityDemandDiff = 0
totalCapacity = 0
totalDemand = 0
totalDemand = sum(totalSalesRegionDemand.values()) * 1000
totalCapacity = sum(b for a, b in productionCapacity) * 1000
print("new yearly demand and total capacity difference: ", totalCapacity - totalDemand)

print(totalSalesRegionDemand)

# distribute the 10% increase across all sales region evenly across chips
# based on their proportion in demand in a region
newSalesRegionDemand = np.copy(salesRegionDemand)


new yearly demand and total capacity difference:  231132.99999999977
{1.0: 45.24300000000001, 2.0: 50.47899999999999, 3.0: 45.65, 4.0: 45.705000000000005, 5.0: 50.225999999999985, 6.0: 44.01100000000001, 7.0: 55.74799999999999, 8.0: 55.80299999999998, 9.0: 45.85900000000001, 10.0: 51.50200000000001, 11.0: 44.81400000000001, 12.0: 45.65, 13.0: 47.60800000000001, 14.0: 52.481, 15.0: 53.394, 16.0: 46.871, 17.0: 50.137999999999984, 18.0: 52.38200000000002, 19.0: 55.187000000000005, 20.0: 49.61, 21.0: 49.83, 22.0: 50.347, 23.0: 54.32899999999999}
[[ 1.    1.    1.18]
 [ 1.    2.    2.17]
 [ 1.    3.    2.82]
 ...
 [23.   28.    2.55]
 [23.   29.    1.57]
 [23.   30.    2.87]]
